In [2]:
-%matplotlib inline
from tabulate import tabulate
import math
import yahoo_finance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

class Portfolio(object):
    '''Define a Portfolio of stocks and their weights.
    input data has to be a dictionary with NYSE stock symbols and the proportion as value.
    '''
    def __init__(self,stocksnweights,equity=0.): 
        if type(stocksnweights) == set or type(stocksnweights) == list:
            self.composition = pd.DataFrame(np.zeros(len(stocksnweights)),
                                            index=[s for s in stocksnweights],
                                            columns=['norm proportion']
                                           )
        elif type(stocksnweights) == dict:
            self.composition = pd.DataFrame(list(stocksnweights.values()),
                                            index=[list(stocksnweights.keys())],
                                            columns=['norm proportion']
                                           )
        else:
            print('Type ERROR')
            return
            
        self._check_comp()
        self.equity = equity
    
    def _check_comp(self):
        s = self.composition['norm proportion'].sum()
        if s == 0.:
            #print("Empty Portfolio Initialized:")
            pass
        elif s != 1.: 
            #print("Sum of weights = ",s)
            #print(" -> Normalizing...")
            for sym in self.composition.index.values:
                self.composition.ix[sym] = self.composition.ix[sym]/s
        #print (tabulate(self.composition, headers=['Symbols','norm proportion']))
        #print("\n")
                
    def load_timeseries(self,time_span):
        '''Put stocks data into a pandas Panel.
        time span has to be a list of strings with format "yyyy-mm-dd", with ordering order "begin", "end".
        '''
        print("Fetching time series data from {0} to {1} :".format(*time_span))
        dict_temp = {}
        for sym in self.composition.index:
            print('Loading "{0}" data...'.format(sym))
            # Fetch
            stock = yf.Share(sym)
            data = pd.DataFrame.from_dict(stock.get_historical(*time_span))
            # Data Cleansing
            del data['Symbol']
            del data['Volume']
            data.set_index("Date", inplace=True)
            data = data.astype("float64")
            data = data.sort_index()
            # Adding features
            data['Return'] = (data['Close']/data['Close'].shift(+1,axis='index')-1.)*100.
            data['Return'].ix[0] = (data['Close'].ix[0]/data['Open'].ix[0]-1.)*100.
            data['Norm_Close'] = data['Close']/data['Open'].ix[0]*100.
            dict_temp[sym] = data
        self.stocks = pd.Panel.from_dict(dict_temp, orient='minor')
        print("Loaded time series of selected shares :")
        print(self.stocks.axes)
        
    def set_shares(self,composition,equity=1):
        if equity != 0: 
            self.equity = equity
        elif self.equity == 0:
            print("Equity set to 1 [arb. units]")
        for symbol in composition.keys():
            self.composition['norm proportion'].ix[symbol] = composition[symbol]
        self._check_comp()
        self._weights = self.composition.copy('deep')
        for sy in self._weights.index:
            self._weights.ix[sy] = self.composition.ix[sy]/self.stocks['Open'][sy].ix[0]
        self._weights.columns = ['weight-per-share']
        self.stocks['Share'] = np.ones((len(self.stocks['Open'].index),len(self._weights.index)))
        self.stocks['Share'] = self.stocks['Share']*self._weights.transpose().ix['weight-per-share']*self.equity
        #print("Calculated weights given opening prices:")
        #print(self.stocks['Open'].ix[0])
        #print(tabulate(self._weights,headers=['Symbol','weight-per-share']))
    

    def get_stats(self):
        #if not self.data.:
        #    print('Warning: data has not been loaded for the current portfolio. Use "get_data" method.')
        #    return
        cols = ['Adj_Close','Close','Open']
        self.data = pd.DataFrame(np.zeros((len(self.stocks[cols[0]].index),len(cols))),
                                 columns=cols,
                                 index=self.stocks[cols[0]].index
                                )
        for col in cols:
            self.data[col] = self.stocks[col].dot(self._weights)*self.equity
        self.data['Return'] = (self.data['Close']/self.data['Close'].shift(+1,axis='index')-1)*100.
        self.data['Return'].ix[0] = (self.data['Close'].ix[0]/self.data['Open'].ix[0]-1)*100. 
        self.data['Norm_Close'] =  self.data['Close']/self.data['Open'].ix[0]*100.
        self.stats = {}
        self.stats['av_daily_return'] = self.data['Return'].mean()
        self.av_daily_return =  self.stats['av_daily_return']
        self.stats['volatility'] = self.data['Return'].std()
        self.volatility = self.stats['volatility']
        self.stats['sharpe'] = math.sqrt(len(self.data.index))*self.av_daily_return/self.volatility
        self.sharpe = self.stats['sharpe']
        return self.av_daily_return, self.volatility, self.sharpe

In [5]:
# Test
sy = {'AAPL','GLD'}
ep = Portfolio(sy)
ts = ["2011-01-01","2011-12-31"]
ep.load_timeseries(ts)
sh = {'AAPL' : 0.65,'GLD' : 0.35}
ep.set_shares(sh,equity=1e+6)

Fetching time series data from 2011-01-01 to 2011-12-31 :
Loading "GLD" data...
Loading "AAPL" data...
Loaded time series of selected shares :
[Index(['Adj_Close', 'Close', 'High', 'Low', 'Open', 'Return', 'Norm_Close'], dtype='object'), Index(['2011-01-03', '2011-01-04', '2011-01-05', '2011-01-06', '2011-01-07',
       '2011-01-10', '2011-01-11', '2011-01-12', '2011-01-13', '2011-01-14',
       ...
       '2011-12-16', '2011-12-19', '2011-12-20', '2011-12-21', '2011-12-22',
       '2011-12-23', '2011-12-27', '2011-12-28', '2011-12-29', '2011-12-30'],
      dtype='object', name='Date', length=252), Index(['AAPL', 'GLD'], dtype='object')]


In [6]:
ep.get_stats()

(0.07650595404501005, 1.1669425588412243, 1.0407490583932348)

In [38]:
ep.stocks.minor_xs('AAPL')

,Adj_Close,Close,High,Low,Open,Return,Norm_Close
Date,,,,,,,
2011-01-03,43.838201,329.570000,330.260002,324.840012,325.640003,1.206853,101.206853
2011-01-04,44.066990,331.290012,332.500000,328.149994,332.439999,0.521896,101.735048
2011-01-05,44.427464,334.000008,334.339989,329.500011,329.549999,0.818013,102.567254
2011-01-06,44.391547,333.729988,335.249996,332.900009,334.719997,-0.080844,102.484334
2011-01-07,44.709458,336.120003,336.349995,331.900013,333.989994,0.716152,103.218278
2011-01-10,45.551451,342.450001,343.229992,337.169987,338.829998,1.883255,105.162142
2011-01-11,45.443707,341.639996,344.959991,339.470013,344.879990,-0.236532,104.913399
2011-01-12,45.813494,344.420006,344.429993,342.000004,343.249992,0.813725,105.767106
2011-01-13,45.981095,345.680008,346.640003,343.850006,345.159996,0.365833,106.154037


In [35]:
#sy =   ['BRCM', 'TXN', 'AMD', 'ADI']
sy=  ['AAPL', 'GOOGL', 'IBM', 'MSFT']
ep = Portfolio(sy)
ep.composition

,norm proportion
AAPL,0
GOOGL,0
IBM,0
MSFT,0


In [36]:
ts = ["2011-01-01","2011-12-31"]
ep.load_timeseries(ts)

Fetching time series data from 2011-01-01 to 2011-12-31 :
Loading "AAPL" data...
Loading "GOOGL" data...
Loading "IBM" data...
Loading "MSFT" data...
Loaded time series of selected shares :
[Index(['Adj_Close', 'Close', 'High', 'Low', 'Open', 'Return', 'Norm_Close'], dtype='object'), Index(['2011-01-03', '2011-01-04', '2011-01-05', '2011-01-06', '2011-01-07',
       '2011-01-10', '2011-01-11', '2011-01-12', '2011-01-13', '2011-01-14',
       ...
       '2011-12-16', '2011-12-19', '2011-12-20', '2011-12-21', '2011-12-22',
       '2011-12-23', '2011-12-27', '2011-12-28', '2011-12-29', '2011-12-30'],
      dtype='object', name='Date', length=252), Index(['AAPL', 'GOOGL', 'IBM', 'MSFT'], dtype='object')]


In [37]:
ep.stocks.minor_xs('MSFT')

,Adj_Close,Close,High,Low,Open,Return,Norm_Close
Date,,,,,,,
2011-01-03,24.378388,27.980000,28.180000,27.920000,28.049999,-0.249551,99.750449
2011-01-04,24.474229,28.090000,28.170000,27.850000,27.940001,0.393138,100.142606
2011-01-05,24.395814,28.000000,28.010000,27.770000,27.900000,-0.320399,99.821750
2011-01-06,25.110262,28.820000,28.850000,27.860001,28.040001,2.928571,102.745102
2011-01-07,24.918581,28.600000,28.740000,28.250000,28.639999,-0.763359,101.960788
2011-01-10,24.587494,28.219999,28.400000,28.040001,28.260000,-1.328675,100.606061
2011-01-11,24.491655,28.110001,28.250000,28.049999,28.200001,-0.389787,100.213911
2011-01-12,24.875017,28.549999,28.590000,28.070000,28.120001,1.565272,101.782531
2011-01-13,24.561357,28.190001,28.389999,28.010000,28.330000,-1.260939,100.499116


In [34]:
bs = -100.
bp = {}
prop = {}
sym_ls=ep.composition.index.values
vals = [el/10 for el in range(11)]
for prop[sym_ls[0]] in vals:
    for prop[sym_ls[1]] in vals:
        for prop[sym_ls[2]] in vals:
            for prop[sym_ls[3]] in vals:
                ep.set_shares(prop)
                if all(el == 0. for el in prop.values()):
                    continue
                m,v,s = ep.get_stats()
                #print('Sharpe =',s)
                if s > bs:
                    bp = ep.composition.to_dict()['norm proportion']
                    bs = s
                    print(bp)
                    print(bs)
print("Optimized portfolio proportions:")
print(bp)
print(bs)

{'MSFT': 1.0, 'IBM': 0.0, 'AAPL': 0.0, 'GOOGL': 0.0}
-0.21311800176908308
{'MSFT': 0.0, 'IBM': 1.0, 'AAPL': 0.0, 'GOOGL': 0.0}
1.1042367350400686
{'MSFT': 0.0, 'IBM': 0.5, 'AAPL': 0.5, 'GOOGL': 0.0}
1.132710273649153
{'MSFT': 0.0, 'IBM': 0.66666666666666663, 'AAPL': 0.33333333333333331, 'GOOGL': 0.0}
1.1510680557267186
{'MSFT': 0.0, 'IBM': 0.69999999999999996, 'AAPL': 0.29999999999999999, 'GOOGL': 0.0}
1.151194242258633
{'MSFT': 0.0, 'IBM': 0.69230769230769229, 'AAPL': 0.30769230769230771, 'GOOGL': 0.0}
1.1512713667155456
Optimized portfolio proportions:
{'MSFT': 0.0, 'IBM': 0.69230769230769229, 'AAPL': 0.30769230769230771, 'GOOGL': 0.0}
1.1512713667155456


In [26]:
ep.set_shares('AMD': 0.0, 'BRCM': 1.0, 'ADBE': 0.0, 'ADI': 0.0)
print(ep.get_stats())
f,ax = plt.subplots(figsize=(10,8))
ep.data['Norm_Close'].plot(ax=ax,title='Portfolio')
ep.stocks['Norm_Close'].plot(ax=ax)

SyntaxError: invalid syntax (<ipython-input-26-a9532481ecac>, line 1)

In [ ]:
ep.stocks['Norm_Close']

In [ ]:
ep.stocks['Norm_Close']

In [ ]:
ep.stocks['Return'].std()

In [ ]:
ep.stocks['Return'].mean()*math.sqrt(255)/ep.stocks['Return'].std()

In [ ]:
ep.set_shares({'MSFT': 0.76923076923076916, 'GOOGL': 0.076923076923076927, 'AAPL': 0.076923076923076927, 'IBM': 0.076923076923076927})
ep.get_stats()
f,ax = plt.subplots(figsize=(10,8))
ep.data['Norm_Close'].plot(ax=ax,label='Portfolio')
ep.stocks['Norm_Close'].plot(ax=ax)

In [ ]:
pd.DataFrame.from_csv()